<a href="https://colab.research.google.com/github/Ujjwalb2/investx/blob/main/lstm_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
pip install ta

In [118]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard

%matplotlib inline
import yfinance as yf

In [119]:
stock_symbol = 'TSLA' #Samsung
start_date = '2015-01-01'
end_date = '2023-01-01'

stock_data = yf.download(stock_symbol, start = start_date, end=end_date,interval='1d')

print(stock_data.head())
stock_data.to_csv('stock_data.csv')

[*********************100%%**********************]  1 of 1 completed

                 Open       High        Low      Close  Adj Close    Volume
Date                                                                       
2015-01-02  14.858000  14.883333  14.217333  14.620667  14.620667  71466000
2015-01-05  14.303333  14.433333  13.810667  14.006000  14.006000  80527500
2015-01-06  14.004000  14.280000  13.614000  14.085333  14.085333  93928500
2015-01-07  14.223333  14.318667  13.985333  14.063333  14.063333  44526000
2015-01-08  14.187333  14.253333  14.000667  14.041333  14.041333  51637500


In [120]:
stocks = pd.read_csv('stock_data.csv')
stocks['Date'] = pd.to_datetime(stocks['Date'])

In [121]:
import ta

In [122]:
stocks['EMA_9'] = ta.trend.ema_indicator(stocks['Close'], window=9)
stocks['EMA_20'] = ta.trend.ema_indicator(stocks['Close'], window=20)
stocks['EMA_55'] = ta.trend.ema_indicator(stocks['Close'], window=55)
stocks['EMA_144'] = ta.trend.ema_indicator(stocks['Close'], window=144)
stocks['RSI'] = ta.momentum.rsi(stocks['Close'], window=14)

In [123]:
stocks

,Date,Open,High,Low,Close,Adj Close,Volume,EMA_9,EMA_20,EMA_55,EMA_144,RSI
0,2015-01-02,14.858000,14.883333,14.217333,14.620667,14.620667,71466000,NaN,NaN,NaN,NaN,NaN
1,2015-01-05,14.303333,14.433333,13.810667,14.006000,14.006000,80527500,NaN,NaN,NaN,NaN,NaN
2,2015-01-06,14.004000,14.280000,13.614000,14.085333,14.085333,93928500,NaN,NaN,NaN,NaN,NaN
3,2015-01-07,14.223333,14.318667,13.985333,14.063333,14.063333,44526000,NaN,NaN,NaN,NaN,NaN
4,2015-01-08,14.187333,14.253333,14.000667,14.041333,14.041333,51637500,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2009,2022-12-23,126.370003,128.619995,121.019997,123.150002,123.150002,166989700,142.705690,158.765993,190.149820,229.033000,19.819923
2010,2022-12-27,117.500000,119.669998,108.760002,109.099998,109.099998,208643400,135.984551,154.035898,187.255184,227.378752,16.564126
2011,2022-12-28,110.349998,116.269997,108.239998,112.709999,112.709999,221070500,131.329641,150.100098,184.592856,225.797114,20.191722
2012,2022-12-29,120.389999,123.570000,117.500000,121.820000,121.820000,221923300,129.427713,147.406756,182.350968,224.362947,28.625201


In [124]:
datelist_train = list(stocks['Date'])

In [125]:
stocks.dropna(inplace=True)

In [126]:
stocks = stocks.drop('Date', axis=1)

In [127]:
stocks

,Open,High,Low,Close,Adj Close,Volume,EMA_9,EMA_20,EMA_55,EMA_144,RSI
143,17.618000,17.859333,17.466667,17.587999,17.587999,41851500,17.644538,17.651400,17.037315,15.838507,50.176253
144,17.512667,17.796000,17.474001,17.785999,17.785999,30519000,17.672830,17.664219,17.064054,15.865369,52.278000
145,17.840000,17.957333,17.674667,17.743334,17.743334,33339000,17.686931,17.671754,17.088314,15.891272,51.771209
146,17.752666,17.780666,17.138000,17.332666,17.332666,38302500,17.616078,17.639460,17.097040,15.911153,47.043928
147,17.334000,17.781334,17.222668,17.752001,17.752001,35287500,17.643263,17.650178,17.120432,15.936544,51.876065
...,...,...,...,...,...,...,...,...,...,...,...
2009,126.370003,128.619995,121.019997,123.150002,123.150002,166989700,142.705690,158.765993,190.149820,229.033000,19.819923
2010,117.500000,119.669998,108.760002,109.099998,109.099998,208643400,135.984551,154.035898,187.255184,227.378752,16.564126
2011,110.349998,116.269997,108.239998,112.709999,112.709999,221070500,131.329641,150.100098,184.592856,225.797114,20.191722
2012,120.389999,123.570000,117.500000,121.820000,121.820000,221923300,129.427713,147.406756,182.350968,224.362947,28.625201


In [128]:
dataset_train=stocks

In [129]:
cols = list(stocks)[1:12]

In [130]:
cols

['High',
 'Low',
 'Close',
 'Adj Close',
 'Volume',
 'EMA_9',
 'EMA_20',
 'EMA_55',
 'EMA_144',
 'RSI']

In [131]:



print('Training set shape == {}'.format(dataset_train.shape))
print('All timestamps == {}'.format(len(datelist_train)))
print('Featured selected: {}'.format(cols))

Training set shape == (1871, 11)
All timestamps == 2014
Featured selected: ['High', 'Low', 'Close', 'Adj Close', 'Volume', 'EMA_9', 'EMA_20', 'EMA_55', 'EMA_144', 'RSI']


In [132]:

# Using multiple features (predictors)

training_set = dataset_train.to_numpy()
print('Shape of training set == {}.'.format(training_set.shape))
training_set

Shape of training set == (1871, 11).


array([[ 17.61800003,  17.85933304,  17.46666718, ...,  17.03731481,
         15.83850686,  50.17625291],
       [ 17.5126667 ,  17.79599953,  17.47400093, ...,  17.06405354,
         15.86536883,  52.27799976],
       [ 17.84000015,  17.95733261,  17.67466736, ...,  17.08831355,
         15.89127179,  51.77120914],
       ...,
       [110.34999847, 116.26999664, 108.23999786, ..., 184.59285571,
        225.79711376,  20.19172176],
       [120.38999939, 123.56999969, 117.5       , ..., 182.350968  ,
        224.36294667,  28.6252012 ],
       [119.94999695, 124.48000336, 119.75      , ..., 180.23771915,
        222.96731983,  29.81752148]])

In [133]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
training_set_scaled = sc.fit_transform(training_set)

sc_predict = StandardScaler()
sc_predict.fit_transform(training_set[:, 0:1])

array([[-0.69295133],
       [-0.69390226],
       [-0.69094716],
       ...,
       [ 0.14421559],
       [ 0.23485481],
       [ 0.23088255]])

In [134]:
# Creating a data structure with 90 timestamps and 1 output
X_train = []
y_train = []

n_future = 60   # Number of days we want top predict into the future
n_past = 90     # Number of past days we want to use to predict the future

for i in range(n_past, len(training_set_scaled) - n_future +1):
    X_train.append(training_set_scaled[i - n_past:i, 0:dataset_train.shape[1] - 1])
    y_train.append(training_set_scaled[i + n_future - 1:i + n_future, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

print('X_train shape == {}.'.format(X_train.shape))
print('y_train shape == {}.'.format(y_train.shape))

X_train shape == (1722, 90, 10).
y_train shape == (1722, 1).


In [135]:
# Import Libraries and packages from Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.optimizers import Adam

In [136]:
# Initializing the Neural Network based on LSTM
model = Sequential()

# Adding 1st LSTM layer
model.add(LSTM(units=64, return_sequences=True, input_shape=(n_past, dataset_train.shape[1]-1)))

# Adding 2nd LSTM layer
model.add(LSTM(units=10, return_sequences=False))

# Adding Dropout
model.add(Dropout(0.25))

# Output layer
model.add(Dense(units=1, activation='linear'))

# Compiling the Neural Network
model.compile(optimizer = Adam(learning_rate=0.01), loss='mean_squared_error')

In [137]:
%%time
es = EarlyStopping(monitor='val_loss', min_delta=1e-10, patience=10, verbose=1)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1)
mcp = ModelCheckpoint(filepath='weights.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True)

tb = TensorBoard('logs')

history = model.fit(X_train, y_train, shuffle=True, epochs=30, callbacks=[es, rlr, mcp, tb], validation_split=0.2, verbose=1,batch_size=256)

Epoch 1/30
6/6 [==============================] - ETA: 0s - loss: 0.1530
Epoch 1: val_loss improved from inf to 1.41642, saving model to weights.h5
6/6 [==============================] - 6s 406ms/step - loss: 0.1530 - val_loss: 1.4164 - lr: 0.0100
Epoch 2/30
6/6 [==============================] - ETA: 0s - loss: 0.0713
Epoch 2: val_loss improved from 1.41642 to 0.63427, saving model to weights.h5
6/6 [==============================] - 1s 246ms/step - loss: 0.0713 - val_loss: 0.6343 - lr: 0.0100
Epoch 3/30
6/6 [==============================] - ETA: 0s - loss: 0.0437
Epoch 3: val_loss improved from 0.63427 to 0.58546, saving model to weights.h5
6/6 [==============================] - 2s 429ms/step - loss: 0.0437 - val_loss: 0.5855 - lr: 0.0100
Epoch 4/30
6/6 [==============================] - ETA: 0s - loss: 0.0336
Epoch 4: val_loss did not improve from 0.58546
6/6 [==============================] - 3s 432ms/step - loss: 0.0336 - val_loss: 0.7267 - lr: 0.0100
Epoch 5/30
6/6 [============

In [138]:
# Generate list of sequence of days for predictions
datelist_future = pd.date_range(datelist_train[-1], periods=n_future, freq='1d').tolist()

'''
Remeber, we have datelist_train from begining.
'''

# Convert Pandas Timestamp to Datetime object (for transformation) --> FUTURE
datelist_future_ = []
for this_timestamp in datelist_future:
    datelist_future_.append(this_timestamp.date())

In [139]:
# Perform predictions
predictions_future = model.predict(X_train[-n_future:])

predictions_train = model.predict(X_train[n_past:])

51/51 [==============================] - 1s 21ms/step


In [140]:
# Inverse the predictions to original measurements

# ---> Special function: convert <datetime.date> to <Timestamp>
def datetime_to_timestamp(x):
    '''
        x : a given datetime value (datetime.date)
    '''
    return datetime.strptime(x.strftime('%Y%m%d'), '%Y%m%d')


y_pred_future = sc_predict.inverse_transform(predictions_future)
y_pred_train = sc_predict.inverse_transform(predictions_train)

PREDICTIONS_FUTURE = pd.DataFrame(y_pred_future, columns=['Open']).set_index(pd.Series(datelist_future))
PREDICTION_TRAIN = pd.DataFrame(y_pred_train, columns=['Open']).set_index(pd.Series(datelist_train[2 * n_past + n_future -1:]))

# Convert <datetime.date> to <Timestamp> for PREDCITION_TRAIN
PREDICTION_TRAIN.index = PREDICTION_TRAIN.index.to_series().apply(datetime_to_timestamp)


ValueError: Length mismatch: Expected 1632 rows, received array of length 1775

In [ ]:
PREDICTIONS_FUTURE

In [ ]:
dataset_train

In [ ]:
# Set plot size
from pylab import rcParams
rcParams['figure.figsize'] = 14, 5

# Plot parameters
START_DATE_FOR_PLOTTING = '2012-06-01'

plt.plot(PREDICTIONS_FUTURE.index, PREDICTIONS_FUTURE['Open'], color='r', label='Predicted Stock Price')
plt.plot(PREDICTION_TRAIN.loc[START_DATE_FOR_PLOTTING:].index, PREDICTION_TRAIN.loc[START_DATE_FOR_PLOTTING:]['Open'], color='orange', label='Training predictions')
# plt.plot(dataset_train.loc[START_DATE_FOR_PLOTTING:].index, dataset_train.loc[START_DATE_FOR_PLOTTING:]['Open'], color='b', label='Actual Stock Price')
plt.plot(stocks['Date'], stocks['Open'], label = 'Actual Stock Price', c = 'blue', alpha=0.5)
# plt.axvline(x = min(PREDICTIONS_FUTURE.index), color='green', linewidth=2, linestyle='--')

plt.grid(which='major', color='#cccccc', alpha=0.5)

plt.legend(shadow=True)
plt.title('Predcitions and Acutal Stock Prices', family='Arial', fontsize=12)
plt.xlabel('Timeline', family='Arial', fontsize=10)
plt.ylabel('Stock Price Value', family='Arial', fontsize=10)
plt.xticks(rotation=45, fontsize=8)
plt.show()